We need to be able to correctly merge DWI series if they're compatible and the user has requested the merge. Here are some example use cases and how they are handled.

## Two scans to be merged

Here there are two scans that have the same phase encoding direction and voxel grid size. The differ in polarity, but that is not important for this test.

In [27]:
from qsiprep.utils.bids import collect_data
subject_data, layout = collect_data('/Users/mcieslak/projects/test_bids_data/pepolar', '1')
# Handle the grouping of multiple dwi files within a session
sessions = layout.get_sessions()
all_dwis = subject_data['dwi']
dwi_groups = []
from collections import defaultdict
if sessions:
    for session in sessions:
        dwi_groups.append([img for img in all_dwis if 'ses-'+session in img])
else:
    dwi_groups = [all_dwis]

def merge_dwis(dwi_files, layout, ignore=[],combine_all_dwis=True):
    parsed_dwis = []
    for ref_file in dwi_files:
        # Find associated sbref, if possible
        metadata = layout.get_metadata(ref_file)

        # Find fieldmaps. Options: (epi|syn)
        fmaps = []
        if 'fieldmaps' not in ignore:
            fmaps = layout.get_fieldmap(ref_file, return_list=True)
            fmap_files = []
            for fmap in fmaps:
                if fmap['type'] == 'epi':
                    fmap_files.append(fmap['epi'])
            if len(fmap_files) > 1:
                print("WARNING MULTIPLE FMAPS FOUND")
                fmap_file = fmap_files[0]
            elif len(fmap_files) == 1:
                fmap_file = fmap_files[0]
            else:
                fmap_file = ""
                
        parsed_dwis.append(
            (ref_file, metadata['PhaseEncodingDirection'], fmap_file))
        
    scans_and_maps = []
    if combine_all_dwis:
        groups = defaultdict(list)
        for ref_file, pe_dir, fmap in parsed_dwis:
            groups[(pe_dir, fmap)].append(ref_file)
        
        if len(groups) > 2:
            LOGGER.critical("Unable to match fieldmaps to DWIs.")
        for k,v in groups.items():
            scans_and_maps.append((v,k))
        if len(groups) == 2:
            LOGGER.warning("Using b0 templates from opposite PEs to unwarp instead of fieldmaps")
    else:
        for ref_file, pe_dir, fmap in parsed_dwis:
            scans_and_maps.append((ref_file, fmap))
                
    # list of [([scans], fieldmap), ([scans], fieldmap)]
    return scans_and_maps

for dwi_group in dwi_groups:
    print(merge_dwis(dwi_group, layout))


[(['/Users/mcieslak/projects/test_bids_data/pepolar/sub-1/dwi/sub-1_dir-AP_dwi.nii.gz'], ('j', '/Users/mcieslak/projects/test_bids_data/pepolar/sub-1/fmap/sub-1_dir-PA_epi.nii.gz')), (['/Users/mcieslak/projects/test_bids_data/pepolar/sub-1/dwi/sub-1_dir-PA_run-1_dwi.nii.gz', '/Users/mcieslak/projects/test_bids_data/pepolar/sub-1/dwi/sub-1_dir-PA_run-2_dwi.nii.gz', '/Users/mcieslak/projects/test_bids_data/pepolar/sub-1/dwi/sub-1_dir-PA_run-3_dwi.nii.gz'], ('j-', '/Users/mcieslak/projects/test_bids_data/pepolar/sub-1/fmap/sub-1_dir-AP_epi.nii.gz'))]


/Users/mcieslak/miniconda3/envs/py3k/lib/python3.6/site-packages/bids/layout/bids_layout.py:116: UserWarning: 'dataset_description.json' file is missing from project root. You may want to set the root path to a valid BIDS project.
  warnings.warn("'dataset_description.json' file is missing from "


In [28]:
%qtconsole